# 1- Data Preparation and Cleaning

In [1]:
# #plotly.offline doesn't push your charts to the clouds
# import plotly.offline as pyo
# #allows us to create the Data and Figure objects
# from plotly.graph_objs import *
# #plotly.plotly pushes your charts to the cloud  
# # import chart_studio.plotly as py
# import plotly.figure_factory as ff
# # import plotly.express as px

# import matplotlib.pyplot as plt
# %matplotlib inline

# # import cufflings to easily plot pandas data frames
# import cufflinks as cf
# # work with cufflinks offline and set its theme
# cf.go_offline()
# cf.set_config_file(theme='pearl')

import pandas as pd
import numpy as np

# from utils import *

import os
import gc
import json
import re
import gzip, pickle

In [2]:
def print_full(x):
    pd.set_option('display.max_colwidth', -1)
    return x

def reset():
    pd.reset_option('display.max_colwidth')

In [3]:
#cat RC_askmen_2018_1.gz RC_askmen_2018_2.gz RC_askmen_2018_3.gz RC_askmen_2018_4.gz RC_askmen_2018_5.gz RC_askmen_2018_6.gz RC_askmen_2018_7.gz RC_askmen_2018_8.gz RC_askmen_2018_9.gz RC_askmen_2018_10.gz > askmen_2018.gz
#cat RC_askwomen_2018_1.gz RC_askwomen_2018_2.gz RC_askwomen_2018_3.gz RC_askwomen_2018_4.gz RC_askwomen_2018_5.gz RC_askwomen_2018_6.gz RC_askwomen_2018_7.gz RC_askwomen_2018_8.gz RC_askwomen_2018_9.gz RC_askwomen_2018_10.gz > askwomen_2018.gz
#cat RC_relationships_2018_1.gz RC_relationships_2018_2.gz RC_relationships_2018_3.gz RC_relationships_2018_4.gz RC_relationships_2018_5.gz RC_relationships_2018_6.gz RC_relationships_2018_7.gz RC_relationships_2018_8.gz RC_relationships_2018_9.gz RC_relationships_2018_10.gz > relationships_2018.gz

# import gzip, pandas as pd, json, pickle
# %matplotlib inline

In [4]:
def read_and_filter(file_path, filter_by='gender', chunksize=100000):
    
    row_count = 0
    df_res = pd.DataFrame()
    for chunk in pd.read_json(file_path, lines=True, chunksize=100000):
        chunk = chunk[~chunk[filter_by].isnull()]
        df_res = pd.concat([df_res, chunk], ignore_index=True)
        
        # report progress
        row_count += chunksize
        if row_count % (3*chunksize) == 0:
            print(f'parsed {row_count} rows')
        
    return df_res


    

In [10]:
%%time
df_askmen = read_and_filter('askMen2018')

parsed 300000 rows
parsed 600000 rows
parsed 900000 rows
parsed 1200000 rows
parsed 1500000 rows
CPU times: user 2min 11s, sys: 10.3 s, total: 2min 22s
Wall time: 1min 45s


In [11]:
df_askmen.memory_usage(deep=True).sum()/1024**2

1083.3202085494995

In [12]:
df_askmen['body'].memory_usage(deep=True)/1024**2

185.96623039245605

In [13]:
df_askmen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601697 entries, 0 to 601696
Data columns (total 52 columns):
author                           601697 non-null object
author_flair_css_class           426450 non-null object
author_flair_text                450696 non-null object
body                             601697 non-null object
can_gild                         601697 non-null bool
controversiality                 601697 non-null int64
created_utc                      601697 non-null int64
distinguished                    28094 non-null object
edited                           601697 non-null int64
gilded                           601697 non-null int64
id                               601697 non-null object
is_submitter                     601697 non-null bool
link_id                          601697 non-null object
parent_id                        601697 non-null object
permalink                        601697 non-null object
retrieved_on                     601697 non-null int64
sco

### Reducing Data Size

Note that the data contains a lot of of `object` data-type columns. Let's convert those to `category` data type, which is much more memory efficient

In [14]:
categoricals = [col for col in df_askmen if df_askmen[col].dtype=='object' \
                if not col in ['mod_reports', 'user_reports', 'author_flair_richtext', 'gildings', 'body']]

data_types = {col: 'category' for col in categoricals}; data_types

{'author': 'category',
 'author_flair_css_class': 'category',
 'author_flair_text': 'category',
 'distinguished': 'category',
 'id': 'category',
 'link_id': 'category',
 'parent_id': 'category',
 'permalink': 'category',
 'subreddit': 'category',
 'subreddit_id': 'category',
 'subreddit_type': 'category',
 'author_flair_template_id': 'category',
 'body_html': 'category',
 'replies': 'category',
 'author_flair_background_color': 'category',
 'author_flair_text_color': 'category',
 'author_flair_type': 'category',
 'rte_mode': 'category',
 'author_fullname': 'category',
 'collapsed_reason': 'category',
 'subreddit_name_prefixed': 'category',
 'gender': 'category'}

In [10]:
# conda install seaborn

In [15]:
df_askmen.columns

Index(['author', 'author_flair_css_class', 'author_flair_text', 'body',
       'can_gild', 'controversiality', 'created_utc', 'distinguished',
       'edited', 'gilded', 'id', 'is_submitter', 'link_id', 'parent_id',
       'permalink', 'retrieved_on', 'score', 'stickied', 'subreddit',
       'subreddit_id', 'subreddit_type', 'author_cakeday', 'no_follow',
       'send_replies', 'author_flair_template_id', 'approved_by', 'banned_by',
       'body_html', 'likes', 'mod_reports', 'num_reports', 'removal_reason',
       'replies', 'report_reasons', 'saved', 'user_reports', 'archived',
       'can_mod_post', 'score_hidden', 'author_flair_background_color',
       'author_flair_richtext', 'author_flair_text_color', 'author_flair_type',
       'rte_mode', 'author_created_utc', 'author_fullname', 'collapsed',
       'collapsed_reason', 'subreddit_name_prefixed', 'gildings',
       'author_patreon_flair', 'gender'],
      dtype='object')

In [16]:
df_askmen['mod_reports'].apply(type).value_counts()

<class 'float'>       520392
<class 'NoneType'>     57508
<class 'list'>         23797
Name: mod_reports, dtype: int64

In [17]:
df_askmen[categoricals] = df_askmen[categoricals].apply(lambda col: col.astype('category'))

In [18]:
df_askmen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601697 entries, 0 to 601696
Data columns (total 52 columns):
author                           601697 non-null category
author_flair_css_class           426450 non-null category
author_flair_text                450696 non-null category
body                             601697 non-null object
can_gild                         601697 non-null bool
controversiality                 601697 non-null int64
created_utc                      601697 non-null int64
distinguished                    28094 non-null category
edited                           601697 non-null int64
gilded                           601697 non-null int64
id                               601697 non-null category
is_submitter                     601697 non-null bool
link_id                          601697 non-null category
parent_id                        601697 non-null category
permalink                        601697 non-null category
retrieved_on                     601697 no

In [19]:
df_askmen.memory_usage(deep=True).sum()/1024**2

623.8097133636475

In [20]:
df_askmen.to_json("askMen2018_clean.json", orient='table')

In [21]:
df_askmen_new = pd.read_json("askMen2018_clean.json", orient='table')

In [22]:
df_askmen_new.head()

,author,author_flair_css_class,author_flair_text,body,can_gild,controversiality,created_utc,distinguished,edited,gilded,...,author_flair_type,rte_mode,author_created_utc,author_fullname,collapsed,collapsed_reason,subreddit_name_prefixed,gildings,author_patreon_flair,gender
0,Mysta02,male,♂,I've never met an attractive woman that (given...,True,0,1514764807,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,m
1,TheCloned,male,♂,Ouch. Being ghosted on really sucks. I can't s...,True,0,1514764904,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,m
2,Shaddow1,male,Male,Yeah exactly. Like I have no specific thoughts...,True,0,1514764939,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,m
3,Libida,female,Username Relevant,I touch my boobs then smell my hands. I have n...,True,0,1514764939,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,f
4,Shaddow1,male,Male,"I’m on a college campus, it sticks around unti...",True,0,1514764972,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,m


In [23]:
df_askmen_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 601697 entries, 0 to 601696
Data columns (total 52 columns):
author                           601697 non-null category
author_flair_css_class           426450 non-null category
author_flair_text                450696 non-null category
body                             601697 non-null object
can_gild                         601697 non-null bool
controversiality                 601697 non-null int64
created_utc                      601697 non-null int64
distinguished                    28094 non-null category
edited                           601697 non-null int64
gilded                           601697 non-null int64
id                               601697 non-null category
is_submitter                     601697 non-null bool
link_id                          601697 non-null category
parent_id                        601697 non-null category
permalink                        601697 non-null category
retrieved_on                     601697 no

In [24]:
df_askmen_new.memory_usage(deep=True).sum()/1024**2

615.252965927124

In [25]:
df_askmen_new.equals(df_askmen)

True

In [26]:
del df_askmen_new

In [27]:
gc.collect()

110

#### Preparing the AskWomen DF

In [28]:
%%time
df_askwomen = read_and_filter('askWomen2018')

parsed 300000 rows
parsed 600000 rows
parsed 900000 rows
parsed 1200000 rows
CPU times: user 1min 43s, sys: 8.01 s, total: 1min 51s
Wall time: 1min 25s


In [29]:
all(df_askwomen.columns == df_askmen.columns)

True

In [30]:
df_askwomen.memory_usage(deep=True).sum()/1024**2

963.7167196273804

In [31]:
df_askwomen[categoricals] = df_askwomen[categoricals].astype('category')

In [33]:
df_askwomen.to_json("askWomen2018_clean.json", orient='table')

In [34]:
gc.collect()

22

#### Preparing the relationships DF

In [36]:
%%time
df_relation = read_and_filter('relationship2018')

parsed 300000 rows
parsed 600000 rows
parsed 900000 rows
parsed 1200000 rows
parsed 1500000 rows
parsed 1800000 rows
parsed 2100000 rows
parsed 2400000 rows
parsed 2700000 rows
parsed 3000000 rows
CPU times: user 4min 43s, sys: 23.3 s, total: 5min 7s
Wall time: 3min 41s


In [37]:
df_relation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704506 entries, 0 to 704505
Data columns (total 52 columns):
author                           704506 non-null object
author_flair_css_class           0 non-null float64
author_flair_text                0 non-null float64
body                             704506 non-null object
can_gild                         704506 non-null bool
controversiality                 704506 non-null int64
created_utc                      704506 non-null int64
distinguished                    105777 non-null object
edited                           704506 non-null int64
gilded                           704506 non-null int64
id                               704506 non-null object
is_submitter                     704506 non-null bool
link_id                          704506 non-null object
parent_id                        704506 non-null object
permalink                        704506 non-null object
retrieved_on                     704506 non-null int64
score     

In [38]:
df_relation.memory_usage(deep=True).sum()/1024**2

1374.600817680359

In [39]:
df_relation[categoricals] = df_relation[categoricals].astype('category')

In [40]:
df_relation.memory_usage(deep=True).sum()/1024**2

955.7396154403687

In [41]:
gc.collect()

44

In [42]:
df_relation.to_json("relationship2018_clean.json", orient='table')

In [43]:
df_relation = pd.read_json('relationship2018_clean.json', orient='table')

In [44]:
df_relation.memory_usage(deep=True).sum()/1024**2

944.2022771835327

In [45]:
df_relation.head()

,author,author_flair_css_class,author_flair_text,body,can_gild,controversiality,created_utc,distinguished,edited,gilded,...,author_flair_type,rte_mode,collapsed,collapsed_reason,subreddit_name_prefixed,author_created_utc,author_fullname,gildings,author_patreon_flair,gender
0,postboxer,NaN,NaN,I think you might feel less guilty if you talk...,True,0,1514764821,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,m
1,Deadlyantigen,NaN,NaN,I doubt she's trying to escape the situation. ...,True,0,1514764829,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,m
2,um_can_you_not,NaN,NaN,Are you going to address my first question?\n\...,True,0,1514764883,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,f
3,sushi594,NaN,NaN,why are you even considering not giving your s...,True,0,1514764948,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,f
4,AutoModerator,NaN,NaN,The above submission has been **removed** beca...,True,0,1514764950,moderator,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,m
